In [32]:
import json
import os
import sys
import argparse
from collections import Counter, defaultdict
import numpy as np
import graph_tool.all as gt



src_dir = os.path.abspath(os.path.join(os.pardir, 'src'))
sys.path[0] = src_dir
from parser.support import ROLES, CREDITS
from parser.my_mongo_db_login import DB_LOGIN_INFO
import parser.support as support
import network.shift_graph_maker as sgm

In [33]:
# from sklearn.metrics import normalized_mutual_info_score
import pandas as pd

In [34]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
with open('../data/raw_data/movies.json') as f:
    movie_file = f.read()
    movie_data = json.loads(movie_file)

In [30]:
role = 'producing'
role_key = role + "_gender_percentage"
all_movies = support.get_movies_df(role_key)
print('Got all_movies')


ValueError: Expected object or value

In [ ]:
#get 90s movies with only producer, id, title, year...
movie_90s = all_movies[(all_movies.year >= 1990) & (all_movies.year < 2000)]
movie_producer_df = movie_90s[['_id', 'producers', 'producing_gender_percentage', 'title', 'year', 'genres']]

In [ ]:
movie_producer_df = movie_producer_df.sort_values('year')

In [ ]:
movie_producer_df.head()

## Community detecton using graph_tool

In [ ]:
movie_producer_df = movie_producer_df.set_index('_id')
movie_producer_dict = movie_producer_df.to_dict(orient='index')

In [ ]:
movie_producer_dict['tt0099622']

### Make graph

## Stochastic block model

In [ ]:
g = sgm.make_graph(movie_producer_dict)

In [ ]:
clabel = g.vp['kind']
state = gt.minimize_blockmodel_dl(g, deg_corr=True, overlap=False, state_args={'clabel':clabel, 'pclabel':clabel})

In [ ]:
state

In [ ]:
state = gt.minimize_nested_blockmodel_dl(g, deg_corr=True, overlap=False, state_args={'clabel':clabel, 'pclabel':clabel},)

In [ ]:
state.get_bs

In [ ]:
gt.draw_hierarchy(state,layout='bipartite',\
              output=None,\
              subsample_edges=1000,\
              hshortcuts=1, hide=0,\
              )

In [ ]:
component = gt.label_largest_component(g)

sum(np.array(component.a))

## giant component community

In [ ]:
# g = make_graph(movie_producer_dict)
# del_list = []
# for v in g.vertices():
#     c = component[v]
#     if c == 0:
#         del_list.append(v)

In [ ]:
# for v in reversed(sorted(del_list)):
#     g.remove_vertex(v)

In [ ]:
# state = gt.minimize_nested_blockmodel_dl(g, deg_corr=True, overlap=False, state_args={'clabel':clabel, 'pclabel':clabel},)
# state

In [ ]:
# gt.draw_hierarchy(state,layout='bipartite',\
#               output=None,\
#               subsample_edges=1000,\
#               hshortcuts=1, hide=0,\
#               )

## remove nodes with one edge

In [ ]:
g = sgm.make_graph(movie_producer_dict)

In [ ]:
one_degree_nodes = []
for v in g.vertices():
    edge_num = len(g.get_out_edges(v))
    if edge_num == 1:
        one_degree_nodes.append(v)

In [ ]:
for v in reversed(sorted(one_degree_nodes)):
    g.remove_vertex(v)

In [ ]:
state = gt.minimize_nested_blockmodel_dl(g, deg_corr=True, overlap=False, state_args={'clabel':clabel, 'pclabel':clabel},)
state.entropy()

In [ ]:
g.num_vertices()

In [ ]:
gt.draw_hierarchy(state,layout='bipartite',\
              output=None,\
              subsample_edges=1500,\
              hshortcuts=1, hide=0,\
              beta=0.5
              )

In [ ]:
state.entropy(), block_state.entropy()

In [ ]:
block_state = gt.minimize_blockmodel_dl(g, deg_corr=True, overlap=False, state_args={'clabel':clabel, 'pclabel':clabel},)


In [ ]:
block_state

In [ ]:
b = block_state.b
print(np.unique(np.array(b.a)))
i = 0
for v in g.vertices():
    print (g.vp['name'][v])
    print(b[v])
    i += 1
    if i == 100:
        break